## Workshop 9
### Parametric building of a roof

#### Code

In [7]:
import math
from pyplasm import *
from larlib import *

In [11]:
def pairList(inputList):
    """
    Given a list L as input, returns a new list formed by couples of consecutive
    elements of L

    Example:
        input:  [e1, e2, e3, ... , eK]
        output: [[e1, e2], [e2, e3], ..., [eK - 1, eK], [eK, e1]]

    Args:
        inputList: the list to process

    Returns:
        outputList: a new list formed by couples of consecutive elements
                    of the input list
    """

    inputLen = len(inputList)
    outputList = []

    for i in range(0, inputLen - 1):
        outputList.append([inputList[i], inputList[i + 1]])

    if (inputLen > 0):
        outputList.append([inputList[inputLen - 1], inputList[0]])

    return outputList



def rotatePoints(directions, angle):
    """
    Given a list of directions L and an angle A as input, returns a list of points
    A direction is represented with a list composed of two element ([point, quadrant]), 
    where point is a list with three integers describing the point coordinates ([x, y, z]), 
    while quadrant is an integer 1 and 4 that determines the rotation for the x and y axis.
    If quadrant is zero then no rotation is applied to the first two coordinates.
    Angle represents the desired rotation angle for the z axis.

    Example:
        input:   [[[1, 1, 1], 2], [[3, 5, 7], 1], [[8, 5, 5], 4]], 0
        output:

    Args:
        directions: a list of directions
        angle:      the angle of rotation in the z axis

    Returns:
       points:      a list of points

    """

    factors = ((1, 1), (-1, 1), (-1, -1), (1, -1))

    points = []

    for point, quadrant in directions:

        if quadrant < 1 or quadrant > 4:
            fX, fY = (0, 0)
        else:
            fX, fY = factors[quadrant - 1]

        points.append(
            [
                point[0] + fX * (30 * math.cos(math.pi / 4)),
                point[1] + fY * (30 * math.sin(math.pi / 4)),
                point[2] + (30 * math.sin(angle))
            ])

    return points + [points[0]]

def ggpl_create_roof(verts, cells, angle, directions):
    """
    Given a list of verts, a list of cells, an angle and a 
    list of directions, returns an HPC model representing a roof
    
    Args:
        verts: list of verts
        cells: list of cells
        angle: rotation angle (z axis)
        directions: list of directions
    
    Returns:
        An HPC model representing a roof
    """
    # utility function for easy texture application
    apply_texture = lambda pol, txt: TEXTURE(txt)(pol)

    # determine "bottom" verts
    bottomVerts = rotatePoints(directions, angle)

    # create slopes and "upper" roof polygon 
    bottom = pairList(bottomVerts)
    top = pairList(verts)
    
    slopes = []
    for i in range(0, len(top)):
        slopes.append(MKPOL([[
                              top[i][0], top[i][1], 
                              bottom[i][0], bottom[i][1]
                             ],
                             [[1, 2, 3, 4]],
                             1]))
        
    roof = STRUCT([MKPOL([bottomVerts, cells, 1])] + slopes + [MKPOL([verts, cells, 1])])
    
    return apply_texture(roof, 'txt/roof.jpg')

#### Input Values

In [12]:
angle = math.pi / 6.0

cells = [[1, 2, 8],
         [2, 3, 8],
         [3, 5, 8], 
         [3, 4, 5],
         [5, 6, 7, 8]]

verts = [[120, 120, 0],
         [120, 75, 0],
         [15, 75, 0],
         [15, 120, 0],
         [45, 120, 0],
         [45, 141, 0],
         [90, 141, 0],
         [90, 120, 0]]

directions = [[verts[0], 1],
              [verts[1], 4],
              [verts[2], 3],
              [verts[3], 2],
              [verts[4], 2],
              [verts[5], 2],
              [verts[6], 1],
              [verts[7], 1],
             ]

#### Output

In [ ]:
VIEW(ggpl_create_roof(verts, cells, angle, directions))

![](images/output.png)